In [2]:
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import time
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import seaborn as sns

DATA_PATH = '../cell_data/data/lognorm-evaluation/eval/max/fibronectin_full/'
RESULT_PATH = './results/2nd_run'

In [10]:
result_paths = [ { 'name': name, 'path': os.path.join(RESULT_PATH, name)} 
                for name in os.listdir(RESULT_PATH) if os.path.isdir(os.path.join(RESULT_PATH, name))]
# experiments = {result['path']: [os.path.join(result['path'], name) for name in os.listdir(result['path'])] for result in result_paths}
experiments = [ {'name': result['name'],'path': result['path'], 
                 'experiments': sorted([os.path.join(result['path'], name) 
                                        for name in os.listdir(result['path']) if os.path.isdir(os.path.join(result['path'], name))])} for result in result_paths]
result_paths, experiments

([{'name': 'cnn', 'path': './results/2nd_run/cnn'},
  {'name': 'mlp', 'path': './results/2nd_run/mlp'},
  {'name': 'encoder', 'path': './results/2nd_run/encoder'},
  {'name': 'mcdcnn', 'path': './results/2nd_run/mcdcnn'},
  {'name': 'fcn', 'path': './results/2nd_run/fcn'},
  {'name': 'resnet', 'path': './results/2nd_run/resnet'},
  {'name': 'inception', 'path': './results/2nd_run/inception'}],
 [{'name': 'cnn',
   'path': './results/2nd_run/cnn',
   'experiments': ['./results/2nd_run/cnn/fibronectin_full_itr_0',
    './results/2nd_run/cnn/fibronectin_full_itr_1',
    './results/2nd_run/cnn/fibronectin_full_itr_2',
    './results/2nd_run/cnn/fibronectin_full_itr_3',
    './results/2nd_run/cnn/fibronectin_full_itr_4']},
  {'name': 'mlp',
   'path': './results/2nd_run/mlp',
   'experiments': ['./results/2nd_run/mlp/fibronectin_full_itr_0',
    './results/2nd_run/mlp/fibronectin_full_itr_1',
    './results/2nd_run/mlp/fibronectin_full_itr_2',
    './results/2nd_run/mlp/fibronectin_full_itr

In [5]:
# obj = experiments[0]
# fig, ax = plt.subplots(2,2, figsize=(15,8))
# for m, experiment in enumerate(obj['experiments']):
#     exp_hist = pd.read_csv(os.path.join(experiment, 'history.csv'))
#     ax[0,0].plot(exp_hist.loss)
#     ax[0,1].plot(exp_hist.val_loss)
#     ax[1,0].plot(exp_hist.accuracy)
#     ax[1,1].plot(exp_hist.val_accuracy)
#     ax[1,0].set_ylim((0,1))
#     ax[1,1].set_ylim((0,1))
# plt.show()

In [28]:
def log(func):
    def wrap(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        
        print('Generated', args[0], round(end-start, 2), 's')
        return result
    return wrap

def get_max_acc_experiment(experiment):
    mx_idx = 0
    max_accs = 0
    for n, exp in enumerate(experiment['experiments']):
        exp_hist = pd.read_csv(os.path.join(exp, 'history.csv'))
        mx = max(exp_hist.val_accuracy)
        if max_accs < mx:
            mx_idx = n
            max_accs = mx
    return mx_idx

def get_predictions(experiment, labels):
    preds = pd.read_csv(os.path.join(experiment, 'true-pred-values.csv'))
    y_pred = np.array(preds.pred)
    pred_labels = [labels[y_pred[n]] for n in range(y_pred.shape[0])]
    return y_pred, pred_labels

def get_test(path, labels):
    x_test = np.load(os.path.join(path, 'X_test.npy'))
    y_test = np.load(os.path.join(path, 'y_test.npy'))
    test_labels = [labels[y_test[n]] for n in range(y_test.shape[0])]
    return x_test, y_test, test_labels

def get_dictionary(path):
    dictionary = pd.read_csv(os.path.join(path, 'dictionary.csv'))
    labels = list(dictionary.iloc[:, 0])
    return labels

In [42]:
@log
def generate_loss_acc_plot(filename, experiment):
    fig, ax = plt.subplots(2,2, figsize=(15,8))
    plt.suptitle(experiment['name'])
    ax[0,0].set_title("Trn loss")
    ax[0,1].set_title("Val loss")
    ax[1,0].set_title("Trn Accuracy")
    ax[1,1].set_title("Val Accuracy")
    for n, exp in enumerate(experiment['experiments']):
        exp_hist = pd.read_csv(os.path.join(exp, 'history.csv'))
        ax[0,0].plot(exp_hist.loss, label=n)
        ax[0,1].plot(exp_hist.val_loss, label=n)
        ax[1,0].plot(exp_hist.accuracy, label=n)
        ax[1,1].plot(exp_hist.val_accuracy, label=n)
    ax[0,0].legend()
    ax[0,1].legend()
    ax[1,0].legend()
    ax[1,1].legend()
    ax[1,0].set_ylim((0,1))
    ax[1,1].set_ylim((0,1))
    plt.savefig(filename)
    plt.close()
    
@log
def generate_tr_tst_plot(filename, experiments):   
    fig, ax = plt.subplots(1,2, figsize=(15,8))
    ax[0].set_title("Trn Accuracy")
    ax[1].set_title("Val Accuracy")
    for exp in experiments:
    #     if exp['name'] not in ['resnet', 'fcn']: 
        mx_idx = get_max_acc_experiment(exp)
        exp_hist = pd.read_csv(os.path.join(exp['experiments'][mx_idx], 'history.csv'))
        ax[0].plot(exp_hist.accuracy, label=exp['name'], alpha=.5)
        ax[1].plot(exp_hist.val_accuracy, label=exp['name'], alpha=.5)
    ax[0].legend()
    ax[1].legend()
    ax[0].set_xlim((0, 400))
    ax[1].set_xlim((0, 400))
    plt.savefig(filename)
    plt.close()

@log
def generate_tst_pred_plot(filename, experiment, x_test, y_test, labels):
    label_count = len(labels)
    cmap = mpl.cm.get_cmap('Set1', label_count)
    for n, exp in enumerate(experiment['experiments']):
        y_pred, _ = get_predictions(exp, labels)
        fig, ax = plt.subplots(5,1, figsize=(15,50))
        fig.suptitle("Test predictions")
        for i in range(label_count):
            for j in range(label_count):
                ax[i].plot([], c=cmap(j), label=labels[j])
            ax[i].legend()
            ax[i].set_title(''.join([labels[i], '-predictions']))

        for n in range(x_test.shape[0]):
            ax[y_test[n]].plot(x_test[n,:], c=cmap(y_pred[n]))
        plt.savefig(os.path.join(exp, 'tst-predictions-types.png'))
        plt.close()

@log
def generate_preds_plot(filename, experiment, x_test, y_test, labels):
    label_count = len(labels)
    cmap = ['r', 'g']
    for n, exp in enumerate(experiment['experiments']):
        y_pred, _ = get_predictions(exp, labels)
        fig, ax = plt.subplots(5,1, figsize=(15,50))
        for i in range(label_count):
            for n, label in enumerate(['false', 'true']):
                ax[i].plot([], c=cmap[n], label=label)
            ax[i].legend()
            ax[i].set_title(''.join([labels[i], '_predictions']))
                
        for n in range(x_test.shape[0]):
            ax[y_test[n]].plot(x_test[n, :], c=cmap[int(y_pred[n] == y_test[n])])
        plt.savefig(os.path.join(exp, 'tst-predictions.png'))
        plt.close()

In [43]:
# Loss-Accuracy plots
print("Generating loss-accuracy plots")
for experiment in experiments:
    generate_loss_acc_plot(os.path.join(experiment['path'], experiment['name']+'-tr-tst-metrics.png'), experiment)
# Total train-test metrics
print("Generating total train-test metrics plot")
generate_tr_tst_plot(os.path.join(RESULT_PATH, 'tr-tst-metrics.png'), experiments)

labels = get_dictionary(DATA_PATH)
x_test, y_test, test_labels = get_test(DATA_PATH, labels)
# successfull_pred = y_test == y_pred

# Test prediction types plots
print("Generating test prediction types plots")
for experiment in experiments:
    generate_tst_pred_plot(experiment['path'], experiment, x_test, y_test, labels)


print("Generating test prediction plots")
for experiment in experiments:
    generate_preds_plot(experiment['path'], experiment, x_test, y_test, labels)

Generating loss-accuracy plots
Generated ./results/2nd_run/cnn/cnn-tr-tst-metrics.png 0.52 s
Generated ./results/2nd_run/mlp/mlp-tr-tst-metrics.png 0.54 s
Generated ./results/2nd_run/encoder/encoder-tr-tst-metrics.png 0.44 s
Generated ./results/2nd_run/mcdcnn/mcdcnn-tr-tst-metrics.png 0.47 s
Generated ./results/2nd_run/fcn/fcn-tr-tst-metrics.png 0.48 s
Generated ./results/2nd_run/resnet/resnet-tr-tst-metrics.png 0.49 s
Generated ./results/2nd_run/inception/inception-tr-tst-metrics.png 0.47 s
Generating total train-test metrics plot
Generated ./results/2nd_run/tr-tst-metrics.png 0.38 s
Generating test prediction types plots
Generated ./results/2nd_run/cnn 14.3 s
Generated ./results/2nd_run/mlp 19.1 s
Generated ./results/2nd_run/encoder 14.47 s
Generated ./results/2nd_run/mcdcnn 14.64 s
Generated ./results/2nd_run/fcn 14.66 s
Generated ./results/2nd_run/resnet 14.82 s
Generated ./results/2nd_run/inception 14.1 s
Generating test prediction plots
Generated ./results/2nd_run/cnn 14.45 s
Gen

In [34]:
exp = experiments[0]['experiments'][0]
preds = pd.read_csv(os.path.join(exp, 'true-pred-values.csv'))
y_pred = np.array(preds.pred)
x_test = np.load(os.path.join(DATA_PATH, 'X_test.npy'))
y_test = np.load(os.path.join(DATA_PATH, 'y_test.npy'))
dictionary = pd.read_csv(os.path.join(DATA_PATH, 'dictionary.csv'))
labels = list(dictionary.iloc[:, 0])
label_count = len(labels)
pred_labels = [labels[y_pred[n]] for n in range(y_pred.shape[0])]
test_labels = [labels[y_test[n]] for n in range(y_test.shape[0])]

In [166]:
# cm = confusion_matrix(test_labels, pred_labels, labels=labels, normalize='true')
# disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
# plt.figure()
# disp.plot(cmap=plt.cm.Blues)
# plt.show()

In [167]:
# plt.hist(x_test[:, -1], bins=100, density=True)
# plt.show()

In [41]:
int(y_pred[1] == y_test[1])

1